In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import LabelEncoder

In [20]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/technocolabs_prosper/prosperLoanData.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
df = pd.read_csv(path)

In [22]:
df['ListingKey'].nunique()

113066

In [23]:
df['ListingNumber'].nunique()

113066

In [24]:
df['LoanKey'].nunique()

113066

In [25]:
df['LoanNumber'].nunique()

113066

In [26]:
df['MemberKey'].nunique()

90831

#dropping duplicates

In [33]:
df.drop_duplicates(subset = 'ListingKey', inplace = True)
df.drop_duplicates(subset = 'ListingNumber', inplace = True)
df.drop_duplicates(subset = 'LoanKey', inplace = True)
df.drop_duplicates(subset = 'LoanNumber', inplace = True)
df.drop_duplicates(subset = 'MemberKey', inplace = True)

In [34]:
df['ListingKey'].nunique()

90831

In [35]:
df['ListingNumber'].nunique()

90831

In [36]:
df['LoanKey'].nunique()

90831

In [37]:
df['LoanNumber'].nunique()

90831

In [38]:
df['MemberKey'].nunique()

90831

In [39]:
df['LoanStatus'] = df['LoanStatus'].replace(['Chargedoff',
 'Past Due (>120 days)'], ['Defaulted','Defaulted'])

In [40]:
df['LoanStatus'] = df['LoanStatus'].replace(['Completed' , 'Past Due (1-15 days)', 
       'Past Due (16-30 days)', 'Past Due (61-90 days)',
       'Past Due (31-60 days)', 'Past Due (91-120 days)','Cancelled'],
       ['Non-Defaulted','Non-Defaulted','Non-Defaulted',
       'Non-Defaulted','Non-Defaulted','Non-Defaulted','Non-Defaulted',])

In [41]:
non_null_ind = []
for i in df['ClosedDate']:
    if pd.isnull(i):
        continue
    else:
      non_null_ind.append(df[df['ClosedDate']==i].index )

In [42]:
for i in non_null_ind:
  df.loc[i,'LoanStatus']='Non-Defaulted'

In [43]:
non_deliquent_ind = []
for i in df['LoanCurrentDaysDelinquent']:
    if i>=180:
      non_deliquent_ind.append(df[df['LoanCurrentDaysDelinquent']==i].index )

In [44]:
for i in non_deliquent_ind:
  df.loc[i,'LoanStatus']='Defaulted'

In [45]:
df.drop(df[df['LoanStatus'] =='Current'].index,inplace=True)

In [46]:
df['LoanStatus'] = df['LoanStatus'].replace([ 'FinalPaymentInProgress'], ['Non-Defaulted'])

In [47]:
category_cols = df.select_dtypes(include=['category','object']).columns
# df[category_cols] = df[category_cols].fillna(df[category_cols].mode().iloc[0])

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
# df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [48]:
# importing sklearn
import sklearn
# importing simpleimputer
from sklearn.impute import SimpleImputer

In [49]:
#numeric columns
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(df[category_cols])
df[category_cols]= imputer.fit_transform(df[category_cols])
#categorical columns

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df[numeric_cols])
df[numeric_cols]= imputer.fit_transform(df[numeric_cols])

#1.Expected ROI

Expected ROI= (Estimated Effective Yield - Estimated Loss) / LoanOriginalAmount)

In [53]:
df['ExpectedROI'] = (df['EstimatedEffectiveYield'] - df['EstimatedLoss']) / df['LoanOriginalAmount'] 
df['ExpectedROI']

0         0.000009
2         0.000030
11        0.000089
12        0.000019
15        0.000031
            ...   
113914    0.000030
113918    0.000013
113923    0.000011
113926    0.000030
113929    0.000044
Name: ExpectedROI, Length: 42789, dtype: float64

percentage

In [54]:
df['ExpectedROI']=df['ExpectedROI']*100
df['ExpectedROI']

0         0.000948
2         0.002978
11        0.008936
12        0.001850
15        0.003115
            ...   
113914    0.002979
113918    0.001277
113923    0.001117
113926    0.002979
113929    0.004372
Name: ExpectedROI, Length: 42789, dtype: float64

#2.Expected EMI

In [51]:
df['Expected_EMI'] = (df['LoanOriginalAmount']*(df['BorrowerRate']/12)*((1+(df['BorrowerRate']/12))**df['Term'])) / (((1+(df['BorrowerRate']/12))**df['Term'])-1)
df['Expected_EMI']

0         330.425160
2         123.322886
11         33.814494
12        137.391471
15        173.707316
             ...    
113914    119.279478
113918    228.508600
113923    300.948902
113926    125.717148
113929    101.246654
Name: Expected_EMI, Length: 42789, dtype: float64

#3.Loan eligibility

In [52]:
df['Eligible'] =df.apply(lambda row: row['MonthlyLoanPayment']<(row['StatedMonthlyIncome']*0.3),axis=1)
df['Eligible']

0         True
2         True
11        True
12        True
15        True
          ... 
113914    True
113918    True
113923    True
113926    True
113929    True
Name: Eligible, Length: 42789, dtype: bool